# Getting Started

In [42]:
import pandas as pd

In [43]:
df = pd.read_csv('data/survey_results_public.csv')

In [44]:
df.shape

(88883, 85)

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88883 entries, 0 to 88882
Data columns (total 85 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Respondent              88883 non-null  int64  
 1   MainBranch              88331 non-null  object 
 2   Hobbyist                88883 non-null  object 
 3   OpenSourcer             88883 non-null  object 
 4   OpenSource              86842 non-null  object 
 5   Employment              87181 non-null  object 
 6   Country                 88751 non-null  object 
 7   Student                 87014 non-null  object 
 8   EdLevel                 86390 non-null  object 
 9   UndergradMajor          75614 non-null  object 
 10  EduOther                84260 non-null  object 
 11  OrgSize                 71791 non-null  object 
 12  DevType                 81335 non-null  object 
 13  YearsCode               87938 non-null  object 
 14  Age1stCode              87634 non-null

In [46]:
# pd.set_option('display.max_columns', df.shape[1])  # configure output to show all columns
pd.set_option('display.max_columns', 5)

Now we can load in the schema CSV file too, and find out what each of the columns in the survey results is all about.

In [47]:
schema = pd.read_csv('data/survey_results_schema.csv')
# pd.set_option('display.max_rows', df.shape[0])   # will allow us to see all 85 rows
pd.set_option('display.max_rows', 8)

Note we can't usually see all 85 rows in the schema, but we've tweaked the `display.max_rows` setting above too.

You can use `.head()` and `.tail()` to limit the number of rows if you don't want that many rows in your output.

In [48]:
schema.head(3)

,Column,QuestionText
0,Respondent,Randomized respondent ID number (not in order ...
1,MainBranch,Which of the following options best describes ...
2,Hobbyist,Do you code as a hobby?


# DataFrame and Series Basics

You can think of how you're accessing a DataFrame with square brackets as being a bit like accessing a dictionary of lists. The keys are like column labels and the lists are like the Series that define the rows.

In [49]:
people = {
    "first": ["Corey", "Jane"],
    "last": ["Schafer", "Doe"]
}

people["first"]

['Corey', 'Jane']

In [50]:
pd.DataFrame(people)

,first,last
0,Corey,Schafer
1,Jane,Doe


See the `Snippets.ipynb` notebook for code showing how to access rows and columns.

Let's look at what we can do with the main data set.

In [51]:
df['Hobbyist']

0        Yes
1         No
2        Yes
3         No
        ... 
88879     No
88880     No
88881     No
88882    Yes
Name: Hobbyist, Length: 88883, dtype: object

In [52]:
df.loc[0, 'Hobbyist']  # first row, Hobbyist column

'Yes'

In [53]:
df.loc
df.loc[0:2, 'Hobbyist':'Employment']

,Hobbyist,OpenSourcer,OpenSource,Employment
0,Yes,Never,The quality of OSS and closed source software ...,"Not employed, and not looking for work"
1,No,Less than once per year,The quality of OSS and closed source software ...,"Not employed, but looking for work"
2,Yes,Never,The quality of OSS and closed source software ...,Employed full-time


# Indexes: set, reset and using

In [54]:
df.set_index('Respondent', inplace=True)  # could also use index_col in read_csv

How do we find the question for a column, given that we have the schema frame?

In [55]:
schema.head(3)

,Column,QuestionText
0,Respondent,Randomized respondent ID number (not in order ...
1,MainBranch,Which of the following options best describes ...
2,Hobbyist,Do you code as a hobby?


In [56]:
schema.set_index('Column', inplace=True)

In [57]:
schema.loc['MgrIdiot']

QuestionText    How confident are you that your manager knows ...
Name: MgrIdiot, dtype: object

In [58]:
schema.sort_index().head(3)

,QuestionText
Column,
Age,What is your age (in years)? If you prefer not...
Age1stCode,At what age did you write your first line of c...
BetterLife,Do you think people born today will have a bet...


# Filtering

Who makes loads of money writing Python, in one of the countries we're most interested in?

In [59]:
countries = ['United States', 'India', 'United Kingdom', 'Germany', 'Canada']
in_country = df['Country'].isin(countries)

In [60]:
does_python = df['LanguageWorkedWith'].str.contains('Python', na=False)

In [61]:
high_salary = (df['ConvertedComp'] > 70000)

In [62]:
df[high_salary & does_python & in_country].head(3)

,MainBranch,Hobbyist,...,SurveyLength,SurveyEase
Respondent,,,,,
22,I am a developer by profession,Yes,...,Appropriate in length,Easy
26,I am a developer by profession,Yes,...,Appropriate in length,Easy
32,I am a developer by profession,No,...,Appropriate in length,Neither easy nor difficult


In [63]:
df.loc[high_salary & does_python, ['Country', 'LanguageWorkedWith', 'ConvertedComp']]

,Country,LanguageWorkedWith,ConvertedComp
Respondent,,,
9,New Zealand,Bash/Shell/PowerShell;C#;HTML/CSS;JavaScript;P...,95179.0
22,United States,Bash/Shell/PowerShell;C++;HTML/CSS;JavaScript;...,103000.0
26,United States,Bash/Shell/PowerShell;C++;C#;HTML/CSS;JavaScri...,114000.0
32,United States,Bash/Shell/PowerShell;HTML/CSS;JavaScript;PHP;...,1100000.0
...,...,...,...
88874,United States,C++;Python;Scala;SQL,2000000.0
88876,United States,Bash/Shell/PowerShell;C#;HTML/CSS;Java;Python;...,180000.0
88879,Finland,Bash/Shell/PowerShell;C++;Python,82488.0
88882,Netherlands,C#;HTML/CSS;Java;JavaScript;PHP;Python,588012.0


# Updating Rows and Columns

In [64]:
df.rename(columns={'ConvertedComp': 'SalaryUSD'}, inplace=True)

In [65]:
df['SalaryUSD']

Respondent
1            NaN
2            NaN
3         8820.0
4        61000.0
          ...   
88601        NaN
88802        NaN
88816        NaN
88863        NaN
Name: SalaryUSD, Length: 88883, dtype: float64

In [66]:
df['Hobbyist'] = df['Hobbyist'].map({'Yes': True, 'No': False})
df

,MainBranch,Hobbyist,...,SurveyLength,SurveyEase
Respondent,,,,,
1,I am a student who is learning to code,True,...,Appropriate in length,Neither easy nor difficult
2,I am a student who is learning to code,False,...,Appropriate in length,Neither easy nor difficult
3,"I am not primarily a developer, but I write co...",True,...,Appropriate in length,Neither easy nor difficult
4,I am a developer by profession,False,...,Appropriate in length,Easy
...,...,...,...,...,...
88601,NaN,False,...,NaN,NaN
88802,NaN,False,...,NaN,NaN
88816,NaN,False,...,NaN,NaN
88863,NaN,True,...,Appropriate in length,Easy
